<img src="NotebookAddons/blackboard-banner.jpg" width="100%" />
<font face="Calibri">
<br>
<font size="5"> <b>An Introduction to Simple SAR Change Detection Methods</b></font>

<br>
<font size="4"> <b> Franz J Meyer; University of Alaska Fairbanks & Josef Kellndorfer, <a href="http://earthbigdata.com/" target="_blank">Earth Big Data, LLC</a> </b> <br>
<img src="NotebookAddons/UAFLogo_A_647.png" style="padding:5px;" width="170" align="right" /></font>

<font size="3">This notebook introduces you to a some popular change detection methods that can be applied on SAR time series data. SAR is an excellent tool for change detection. As SAR sensors are weather and illumination independent, and as SAR's carry their own illumination source (active sensor), differences between repeated images are a direct indication of changes on the surface. This fact is exploited by the change detection methods introduced below. 
    
The exercise is done in the framework of *Jupyter Notebooks*. The Jupyter Notebook environment is easy to launch in any web browser for interactive data exploration with provided or new training data. Notebooks are comprised of text written in a combination of executable python code and markdown formatting including latex style mathematical equations. Another advantage of Jupyter Notebooks is that they can easily be expanded, changed, and shared with new data sets or newly available time series steps. Therefore, they provide an excellent basis for collaborative and repeatable data analysis. <br>

<b>This notebook covers the following data analysis concepts:</b>

- Time series metrics  95$^{th}$ and 5$^{th}$ percentile difference thresholding
- Time series coefficient of variation thresholding
- Log Ratio-based change detection from image pairs

<font color='rgba(200,0,0,0.2)'> <b>Note</b>:</font> The next notebook will add Time Series Change Point Detection as an additional useful method.

</font>


</font>

<hr>
<font face="Calibri" size="5" color='rgba(200,0,0,0.2)'> <b>Important Note about JupyterHub</b> </font>
<br><br>
<font face="Calibri" size="3"> <b>Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.</b> </font>


<hr>
<font face="Calibri">

<font size="5"> <b> 0. Importing Relevant Python Packages </b> </font>

<font size="3">In this notebook we will use the following scientific libraries:
<ol type="1">
    <li> <b><a href="https://pandas.pydata.org/" target="_blank">Pandas</a></b> is a Python library that provides high-level data structures and a vast variety of tools for analysis. The great feature of this package is the ability to translate rather complex operations with data into one or two commands. Pandas contains many built-in methods for filtering and combining data, as well as the time-series functionality. </li>
    <li> <b><a href="https://www.gdal.org/" target="_blank">GDAL</a></b> is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.</li>
    <li> <b><a href="http://www.numpy.org/" target="_blank">NumPy</a></b> is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects. </li>
    <li> <b><a href="https://matplotlib.org/index.html" target="_blank">Matplotlib</a></b> is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs. Moreover, many popular plotting libraries are designed to work in conjunction with matplotlib. </li>
    <li> The <b><a href="https://www.pydoc.io/pypi/asf-hyp3-1.1.1/index.html" target="_blank">asf-hyp3 API</a></b> provides useful functions and scripts for accessing and processing SAR data via the Alaska Satellite Facility's Hybrid Pluggable Processing Pipeline, or HyP3 (pronounced "hype"). </li>
<li><b><a href="https://www.scipy.org/about.html" target="_blank">SciPY</a></b> is a library that provides functions for numerical integration, interpolation, optimization, linear algebra and statistics. </li>

</font>

<font face="Calibri" size="3"> Our first step is to <b>import them:</b> </font>

In [ ]:
import os # for chdir, getcwd, path.exists
import datetime # for date

import pandas as pd # for DatetimeIndex
from osgeo import gdal # for Info

import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib.patches as patches  # for Rectangle
from matplotlib import rc
import numpy as np

from IPython.display import HTML

from asf_notebook import path_exists
from asf_notebook import asf_unzip
from asf_notebook import new_directory

<font face="Calibri" size="3"><b>Setup matplotlib plotting</b> inside the notebook:</font>

In [ ]:
%matplotlib inline 

<hr>
<font face="Calibri">

<font size="5"> <b> 1. Load Data Stack</b> </font> <img src="NotebookAddons/Deforest-MadreDeDios.jpg" width="350" style="padding:5px;" align="right" /> 

<font size="3"> This notebook will again be using a 78-image deep dual-polarization C-band SAR data stack over Madre de Dios in Peru to demonstrate how to create color composites from multi temporal and dual-polarization SAR data and how to derive higher level parameters such as the multi-temporal mean, the coefficient of variation, and others. The C-band data were acquired by ESA's Sentinel-1 SAR sensor constellation and are available to you through the services of the <a href="https://www.asf.alaska.edu/" target="_blank">Alaska Satellite Facility</a>. 

The site in question is interesting as it has experienced extensive logging over the last 10 years (see image to the right; <a href="https://blog.globalforestwatch.org/" target="_blank">Monitoring of the Andean Amazon Project</a>). Since the 1980s, people have been clearing forests in this area for farming, cattle ranching, logging, and (recently) gold mining. Creating RGB color composites is an easy way to visualize ongoing changes in the landscape.
</font></font>
<br><br>
<font face="Calibri" size="3">Before we get started, let's first <b>create a working directory for this analysis and change into it:</b> </font>

In [ ]:
path = "/home/jovyan/notebooks/SAR_Training/English/Ecosystems/data_Ex2-4_S1-MadreDeDios"
new_directory(path)
os.chdir(path)
print(f"Current working directory: {os.getcwd()}")

<font face="Calibri" size="3">We will <b>retrieve the relevant data</b> from an <a href="https://aws.amazon.com/" target="_blank">Amazon Web Service (AWS)</a> cloud storage bucket <b>using the following command</b>:</font></font>

In [ ]:
time_series_path = 's3://asf-jupyter-data/MadreDeDios.zip'
time_series = os.path.basename(time_series_path)
!aws s3 cp $time_series_path $time_series

<font face="Calibri" size="3"> Now, let's <b>unzip the file (overwriting previous extractions) and clean up after ourselves:</b> </font>

In [ ]:
if path_exists(time_series):
    asf_unzip(os.getcwd(), time_series)
    os.remove(time_series)

<br>
<font face="Calibri" size="5"> <b> 2. Define Some Python Helper Functions for this Notebook </b> </font> 
<br><br>
<font face="Calibri" size="3">We are defining two helper functions for this notebook:

- **create_geotiff()** to write out images
- **timeseries_metrics()** to compute various metrics from a time series data stack</font>

In [ ]:
def create_geotiff(name, array, data_type, ndv, bandnames=None, ref_image=None, 
                  geo_t=None, projection=None):
    # If it's a 2D image we fake a third dimension:
    if len(array.shape) == 2:
        array = np.array([array])
    if ref_image == None and (geo_t == None or projection == None):
        raise RuntimeWarning('ref_image or settings required.')
    if bandnames != None:
        if len(bandnames) != array.shape[0]:
            raise RuntimeError('Need {} bandnames. {} given'
                               .format(array.shape[0],len(bandnames)))
    else:
        bandnames = ['Band {}'.format(i+1) for i in range(array.shape[0])]
    if ref_image != None:
        refimg = gdal.Open(ref_image)
        geo_t = refimg.GetGeoTransform()
        projection = refimg.GetProjection()
    driver = gdal.GetDriverByName('GTIFF')
    array[np.isnan(array)] = ndv
    DataSet = driver.Create(name, array.shape[2], 
                            array.shape[1], array.shape[0], 
                            data_type)
    DataSet.SetGeoTransform(geo_t)
    DataSet.SetProjection(projection)
    for i, image in enumerate(array, 1):
        DataSet.GetRasterBand(i).WriteArray( image )
        DataSet.GetRasterBand(i).SetNoDataValue(ndv)
        DataSet.SetDescription(bandnames[i-1])
    DataSet.FlushCache()
    return name

In [ ]:
def timeseries_metrics(raster, ndv=0): 
    # Make us of numpy nan functions
    # Check if type is a float array
    if not raster.dtype.name.find('float') > -1:
        raster = raster.astype(np.float32)
    # Set ndv to nan
    if ndv != np.nan:
        raster[np.equal(raster, ndv)] = np.nan
    # Build dictionary of the metrics
    tsmetrics = {}
    rperc = np.nanpercentile(raster, [5, 50, 95], axis=0)
    tsmetrics['mean'] = np.nanmean(raster, axis=0)
    tsmetrics['max'] = np.nanmax(raster, axis=0)
    tsmetrics['min'] = np.nanmin(raster, axis=0)
    tsmetrics['range'] = tsmetrics['max'] - tsmetrics['min']
    tsmetrics['median'] = rperc[1]
    tsmetrics['p5'] = rperc[0]
    tsmetrics['p95'] = rperc[2]
    tsmetrics['prange'] = rperc[2] - rperc[0]
    tsmetrics['var'] = np.nanvar(raster, axis=0)
    tsmetrics['cov'] = tsmetrics['var'] / tsmetrics['mean']
    return tsmetrics

<br>
<font face="Calibri" size="5"> <b> 3. Define Data Directory and Path to VRT </b> </font> 
<br><br>
<font face="Calibri" size="3"><b>Create a variable containing the VRT filename and the image acquisition dates:</b></font>

In [ ]:
!gdalbuildvrt -separate raster_stack.vrt tiffs/*_VH.tiff
image_file_VH = "raster_stack.vrt"

<font face="Calibri" size="3"><b>Create an index of timedelta64 data with Pandas:</b></font>

In [ ]:
!ls tiffs/*_VH.tiff | sort | cut -c 7-21 > raster_stack_VH.dates
datefile_VH = 'raster_stack_VH.dates'
with open(datefile_VH) as f:
    dates_VH = f.readlines()
tindex_VH = pd.DatetimeIndex(dates_VH)

<font face="Calibri" size="3"><b>Print the bands and dates for all images in the virtual raster table (VRT):</b></font>

In [ ]:
print(f"Bands and dates for {image_file_VH}")
for i, t in enumerate(tindex_VH):
    print("{:4d} {}".format(i+1, t.date()), end=' ')
    if (i+1) % 5 == 1:
        print()

<hr>
<br>
<font face="Calibri" size="5"> <b> 4. Create a Time Series Animation to get an Idea of the Dynamics at the Site </b> </font>

<font face="Calibri" size="4"> <b> 4.1 Load Time Series Stack </b> </font>

<font face="Calibri" size="3">Now we are ready to create a time series animation from the calibrated SAR data.
<br><br>
<b>First, create a raster from band 0 and a raster stack from all the images:</b>
</font> 

In [ ]:
img = gdal.Open(image_file_VH)
band = img.GetRasterBand(1)
raster0 = band.ReadAsArray()
band_number = 0 # Needed for updates
rasterstack_VH = img.ReadAsArray()

<font face="Calibri" size="3"><b>Print the bands, pixels, and lines:</b></font>

In [ ]:
print(f"Number of  bands: {img.RasterCount}")
print(f"Number of pixels: {img.RasterXSize}")
print(f"Number of  lines: {img.RasterYSize}")

<br>
<font face="Calibri" size="4"> <b> 4.2 Calibration and Data Conversion between dB and Power Scales </b> </font>

<font face="Calibri" size="3"> <font color='rgba(200,0,0,0.2)'> <b>Note, that if your data were generated by HyP3, this step is not necessary!</b> HyP3 performs the full data calibration and provides you with calibrated data in power scale. </font>
    
If, your data is from a different source, however, calibration may be necessary to ensure that image gray values correspond to proper radar cross section information. 

Calibration coefficients for SAR data are often defined in the decibel (dB) scale due to the high dynamic range of the imaging system. For the L-band ALOS PALSAR data at hand, the conversion from uncalibrated DN values to calibrated radar cross section values in dB scale is performed by applying a standard **calibration factor of -83 dB**. 
<br> <br>
$\gamma^0_{dB} = 20 \cdot log10(DN) -83$

The data at hand are radiometrically terrain corrected images, which are often expressed as terrain flattened $\gamma^0$ backscattering coefficients. For forest and land cover monitoring applications $\gamma^o$ is the preferred metric.

<b>To apply the calibration constant for your data and export in *dB* scale, uncomment the following code cell</b>: </font> 

In [ ]:
 #caldB=20*np.log10(rasterstack)-83

<font face="Calibri" size="3"> While **dB**-scaled images are often "visually pleasing", they are often not a good basis for mathematical operations on data. For instance, when we compute the mean of observations, it makes a difference whether we do that in power or dB scale. Since dB scale is a logarithmic scale, we cannot simply average data in that scale. 
    
Please note that the **correct scale** in which operations need to be performed **is the power scale.** This is critical, e.g. when speckle filters are applied, spatial operations like block averaging are performed, or time series are analyzed.

To **convert from dB to power**, apply: $\gamma^o_{pwr} = 10^{\frac{\gamma^o_{dB}}{10}}$ </font>

In [ ]:
#calPwr=np.power(10.,caldB/10.)

<br>
<font face="Calibri" size="4"> <b> 4.3 Create Time Series Animation </b> </font>

<font face="Calibri" size="3"><b>Create and move into a directory in which to store our plots and animations:</b></font> 

In [ ]:
product_path = 'plots_and_animations'
new_directory(product_path)

<font face="Calibri" size="3">Now we can <b>create the information needed to animate our data:</b></font> 

In [ ]:
%%capture 
fig = plt.figure(figsize=(14, 7))
ax = fig.subplots()
ax.axis('off')
vmin = np.percentile(rasterstack_VH.flatten(), 5)
vmax = np.percentile(rasterstack_VH.flatten(), 95)

r0dB = 20 * np.log10(raster0) - 83

im = ax.imshow(raster0, cmap='gray', vmin=vmin, vmax=vmax)
ax.set_title("{}".format(tindex_VH[0].date()))

def animate(i):
    ax.set_title("{}".format(tindex_VH[i].date()))
    im.set_data(rasterstack_VH[i])

# Interval is given in milliseconds
ani = animation.FuncAnimation(fig, animate, frames=rasterstack_VH.shape[0], interval=400)

<font face="Calibri" size="3"><b>Configure matplotlib's RC settings for the animation:</b></font> 

In [ ]:
rc('animation', embed_limit=40971520.0)  # We need to increase the limit maybe to show the entire animation

<font face="Calibri" size="3"><b>Create a javascript animation of the time-series running inline in the notebook:</b></font> 

In [ ]:
HTML(ani.to_jshtml())

<font face="Calibri" size="3"><b>Delete the dummy png</b> that was saved to the current working directory while generating the javascript animation in the last code cell.</font> 

In [ ]:
try:
    os.remove('None0000000.png')
except FileNotFoundError:
    pass

<font face="Calibri" size="3"><b>Save the animation (animation.gif):</b> </font> 

In [ ]:
ani.save(f'{product_path}/animation_VH.gif', writer='pillow', fps=2)

<hr>
<br>
<font face="Calibri" size="5"> <b> 5. Computation and Visualization of Time Series Metrics</b> </font>

<font face="Calibri" size="3">Once a time-series was constructed, we can compute <b>a set of metrics</b> that will aid us later in applications such as <i>change detection and active agriculture detection</i>. In the next code cells, we will compute the following variables for each pixel in the stack:

- Mean 
- Median
- Maximum
- Minimum
- Range (Maximum - Minimum)
- 5th Percentile
- 95th Percentile
- PRange (95th - 5th Percentile)
- Variance
- Coefficient of Variation (Variance/Mean)

<hr>
First, we <b>mask out pixels</b> without relevant information to be unbiased in statical number we calculate later. Then we <b>calculate the time series metrics</b>:
</font> 

In [ ]:
mask = rasterstack_VH == 0
rasterPwr = np.ma.array(rasterstack_VH, mask=mask, dtype=np.float32)

In [ ]:
metrics = timeseries_metrics(rasterPwr.filled(np.nan), ndv=np.nan)

In [ ]:
metric_keys = metrics.keys()
print(metric_keys)

<font face="Calibri" size="3">Let's look at the histograms for the time series variance and coeficient of variation to aid displaying those images:</font> 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 4))
ax[0].hist(metrics['var'].flatten(), bins=100, range=(0, 0.0001))
ax[1].hist(metrics['cov'].flatten(), bins=100, range=(0, 0.004))
_ = ax[0].set_title('Variance')
_ = ax[1].set_title('Coefficient of Variation')

<font face="Calibri" size="3">We use thresholds determined from those histograms to set the scaling in the time series visualization. For the backscatter metrics we choose a typical range appropriate for this ecosystem, the C-band radar sensor, and the VH polarization. A typical range is -30 dB (0.0001) to -10 dB (0.1).</font> 

In [ ]:
# List the metrics keys you want to plot
fig = plt.figure(figsize=(16, 40))
for i, key in enumerate(metric_keys):
    ax = fig.add_subplot(5, 2, i+1)
    if key == 'var':
        vmin, vmax = (0.0, 0.0001)
    elif key == 'cov': 
        vmin, vmax = (0.0, 0.002)
    else:
        vmin, vmax = (0.0, 0.1)
    ax.imshow(metrics[key], vmin=vmin, vmax=vmax, cmap='gray')
    ax.set_title(key.upper())
    ax.axis('off')

<hr>
<br>
<font face="Calibri" size="5"> <b> 6. Some Popular SAR Change Detection Methods</b> </font>

<font face="Calibri" size="3">This section will introduce you to the following popular and simple change detection methods:

- Time series metrics  95$^{th}$ and 5$^{th}$ percentile difference thresholding
- Time series coefficient of variation thresholding
- Log Ratio-based change detection from image pairs
</font> 

<hr>
<font face="Calibri" size="4"> <b> 6.1 Change Detection with the Percentile Difference and the Variance Threshold Method</b> </font>

<font face="Calibri" size="3">In this method we find thresholds on the <b>95$^{th}$ and 5$^{th}$ percentile difference</b> or the <b>temporal pixel-by-pixel gray value cariance</b>. Let's start with the 95$^{th}$ and 5$^{th}$ percentile difference. The advantage to look at percentiles verus maximum minus minimum is that outliers and extremas in the time series are not influencing the result.

For our example, the historgram and the Cumulative Distribution Function (CDF) of the 95$^{th}$ and 5$^{th}$ percentile difference image looks like this:
</font> 

In [ ]:
plt.rcParams.update({'font.size': 14})
fig = plt.figure(figsize=(14, 4)) # Initialize figure with a size
ax1 = fig.add_subplot(121)  # 121 determines: 2 rows, 2 plots, first plot
ax2 = fig.add_subplot(122)

h = ax1.hist(metrics['range'].flatten(), bins=200, range=(0, 0.1))
ax1.xaxis.set_label_text('Percentile Range Metric')
ax1.set_title('Histogram')

n, bins, patches = ax2.hist(metrics['range'].flatten(), 
                            bins=200, range=(0, 0.1), 
                            cumulative='True', density='True', 
                            histtype='step', label='Empirical')
ax2.xaxis.set_label_text('Percentile Range Metric')
ax2.set_title('CDF')
plt.savefig(os.path.join(product_path, 'thresh_percentilerange_histogram.png'), 
            dpi=200, transparent='true')

outind = np.where(n > 0.95)
threshind = np.min(outind)
thresh = bins[threshind]
ax1.axvline(thresh, color='red')
_ = ax2.axvline(thresh, color='red')

<br>
<font face="Calibri" size="3">Let's visualize the 5% of all pixels with the largest (95th - 5th percentile) difference in the time series. We will refer to the pixels (x,y) that exceed this threshold $t$ as likely <b>change pixels (cp)</b>:

${cp}_{x,y} = P_{x,y}^{95th} - P_{x,y}^{5th} > t$ 

If we define $t$ to correspond to the 5% of pixels with highest (95th - 5th percentile) difference, the image looks like:</font> 

In [ ]:
plt.figure(figsize=(8, 8))
mask = metrics['range'] < thresh # For display we prepare the inverse mask
maskpdiffrange=~mask # Store this for later output
plt.imshow(mask, cmap='gray')
_ = plt.title('Threshold Classifier on Range > %1.3f' % thresh)
plt.savefig(os.path.join(product_path,'changes_percentilerange_threshold.png'), dpi=200, transparent='true')

<hr>
<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE</u> </font></b> 

<font face="Calibri" size="3"> Discuss what you see in this figure. What are the black areas? What are the white areas? What kind of changes may be included in this map?
</font>
</div>
<br>
<hr>

<br>
<font face="Calibri" size="3">Instead of applying a threshold on the 95th - 5th percentile difference data, we can also attempt to threshold other metrics. The <b>Variance</b> variable seems a useful indicator for change as it identifies pixels for which radar brightness has changed strongly within the time series. Hence, in the following we use this metric for change identification according to:

${cp}_{x,y} = \sigma^2 > t$ 

with $t=CDF_{\sigma^2} > 0.95$ (5% pixels with highest variance):</font> 

In [ ]:
plt.rcParams.update({'font.size': 14})
fig = plt.figure(figsize=(14, 4)) # Initialize figure with a size
ax1 = fig.add_subplot(121)  # 121 determines: 2 rows, 2 plots, first plot
ax2 = fig.add_subplot(122)

h = ax1.hist(metrics['var'].flatten(), bins=200, range=(0, 0.0001))
ax1.xaxis.set_label_text('Variance Metric')
ax1.set_title('Histogram')

n, bins, patches = ax2.hist(metrics['var'].flatten(), bins=200, range=(0, 0.0001), cumulative='True', density='True', histtype='step', label='Empirical')
ax2.xaxis.set_label_text('Variance Metric')
ax2.set_title('CDF')
plt.savefig(os.path.join(product_path,'thresh_var_histogram.png'), dpi=200, transparent='true')

outind = np.where(n > 0.95)
threshind = np.min(outind)
threshvar = bins[threshind]
ax1.axvline(threshvar,color='red')
_ = ax2.axvline(threshvar,color='red')

In [ ]:
plt.figure(figsize=(8, 8))
mask = metrics['var'] < threshvar # For display we prepare the inverse mask
maskvar = ~mask # Store this for later output
plt.imshow(mask, cmap='gray')
_ = plt.title('Threshold Classifier on Variance > %1.5f' % threshvar )
plt.savefig(f'{product_path}/changes_var_threshold.png', dpi=200, transparent='true')

<hr>
<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE</u> </font></b> 

<font face="Calibri" size="3"> Discuss what you see in this figure. What kind of changes are detected? How does this change map compare to the previous one?
</font>
</div>
<br>
<hr>

<br>
<font face="Calibri" size="4"> <b> 6.2 Change Detection with the Coefficient of Variation Method </b> </font>

<font face="Calibri" size="3">We can also set a threshold $t$ for the <b>coefficient of variation image</b>
to classify change in the time series:
    
${cp}_{x,y} = \frac{\sigma_{x,y}^2}{\overline{X}_{x,y}} > t$ 

Let's look at the histogram and the Cumulative Distribution Function (CDF) of the coefficient of variation:</font> 

In [ ]:
plt.rcParams.update({'font.size': 14})
fig = plt.figure(figsize=(14, 4)) # Initialize figure with a size
ax1 = fig.add_subplot(121)  # 121 determines: 2 rows, 2 plots, first plot
ax2 = fig.add_subplot(122)

h = ax1.hist(metrics['cov'].flatten(), bins=200, range=(0, 0.002))
ax1.xaxis.set_label_text('Coefficient of Variation Metric')
ax1.set_title('Histogram')

n, bins, patches = ax2.hist(metrics['cov'].flatten(), bins=200, range=(0, 0.002), cumulative='True', density='True', histtype='step', label='Empirical')
ax2.xaxis.set_label_text('Coefficient of Variation Metric')
ax2.set_title('CDF')
plt.savefig(os.path.join(product_path,'thresh_cov_histogram.png'), dpi=200, transparent='true')

outind = np.where(n > 0.95)
threshind = np.min(outind)
threshcov = bins[threshind]
ax1.axvline(threshcov,color='red')
_ = ax2.axvline(threshcov,color='red')

<br>
<font face="Calibri" size="3">With a threshold of $t=CDF_{cov^2} > 0.95$ (5% pixels with highest variance) the change pixels would look like the following image:</font> 

In [ ]:
plt.figure(figsize=(8,8))
mask = metrics['cov'] < threshcov # For display we prepare the inverse mask
maskcov = ~mask # Store this for later output
plt.imshow(mask, cmap='gray')
_ = plt.title('Threshold Classifier on Coefficient of Variation > %1.3f' % threshcov )
plt.savefig(f"{product_path}/changes_CV_threshold.png", dpi=200, transparent='true')

<hr>
<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE</u> </font></b> 

<font face="Calibri" size="3"> Discuss what you see in this figure. What kind of changes are detected? How is this change map different from the previous two? What are the similarities?
</font>
</div>
<br>
<hr>

<br>
<font face="Calibri" size="4"> <b> 6.3 Change Detection with the Log Ratio Method </b> </font>

<font face="Calibri" size="3">In this method, we compare two images from the same season in different years. First we <b>look at global means of the backscatter images building a time series object of acquisition dates and global image means of backscatter.</b></font> 

In [ ]:
tsmean = 10*np.log10(np.nanmean(rasterPwr.filled(np.nan), axis=(1, 2)))

<br>
<font face="Calibri" size="3">We make a time series object to list the dates, mean backscatter in dB and band index number for the rasterPwr array:</font> 

In [ ]:
ts = pd.Series(tsmean, index=tindex_VH)
for i in range(len(ts)):
    print(i, ts.index[i].date(), ts[i])

<br>
<font face="Calibri" size="3"><font color='rgba(200,0,0,0.2)'> <b>To compare two dates for change detection with the log ratio approach we pick two dates of relatively similar backscatter and from similar times of the year.</b></font> Two such candidate dates are:</font> 

For our cross-polarized data

- 2017-09-10 -13.0938015 (index 21)
- 2018-09-17 -13.282042 (index 51)

In [ ]:
# # Cross-polarized
cross_pol_ref = rasterPwr[21]   # Reference Image
cross_pol_new = rasterPwr[51]  # New Image

<br>
<font face="Calibri" size="3">The Log ratio between the images is 

$r  = log_{10}(\frac{X_i}{X_r})$
</font>

In [ ]:
r = np.log10(cross_pol_new / cross_pol_ref)

<br>
<font face="Calibri" size="3"><b>To find a threshold for change</b>, we can display the absolute ratio image $abs(r)$ and the historgram of $r$. We adjust the scale factors for the display to enhance visualization of change areas with largest backscatter change between this image pair. Brighter values show larger change. </font>

In [ ]:
# Display r
fig, ax = plt.subplots(2, 2, figsize=(16, 16))
ax[0][0].axis('off')
ax[0][1].axis('off')
ax[1][0].axis('off')
ax[0][0].set_title('C-VH')
ax[0][0].imshow(cross_pol_ref, vmin=0, vmax=0.1, cmap='gray')
ax[0][1].imshow(cross_pol_new, vmin=0, vmax=0.1, cmap='gray')
ax[1][0].imshow(np.abs(r), vmin=0, vmax=0.3, cmap='gray')
_ = ax[1][1].hist(r.flatten(), bins=100, range=(-0.4, 0.4))

<br>
<font face="Calibri" size="3">Let's define change pixels as those falling outside the range of <b>three</b> times the standard deviation of the ration image $\sigma_r$ from the image mean $\bar{r}$:

${cp}_{x,y} = (r_{x,y} < \overline{r} - 3\sigma_r) \ \textrm{or} \ (r_{x,y} > \overline{r} + 3\sigma_r)$ 

We are using the numpy masking to set the non-changing pixels inside the range:</font>

In [ ]:
stddev = np.std(r)
thres = 3 * stddev
mask = np.logical_and(r>-1*thres, r<thres)
masklr = ~mask

<br>
<font face="Calibri" size="3">Let's display pixels that fall outside 3 times the standard deviation.</font>

In [ ]:
fig,ax = plt.subplots(figsize=(8,16))
ax.imshow(mask, cmap='gray')
ax.xaxis.set_ticks([])
ax.yaxis.set_ticks([])
_ = ax.set_title('Log Ratio Classifier of the September 2017/2018 Log Ratio Images')
plt.savefig(f"{product_path}/changes_LR_threshold.png", dpi=200, transparent='true')

<hr>
<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE</u> </font></b> 

<font face="Calibri" size="3"> Discuss what you see in this figure. What kind of changes are detected? How is this change map different from the previous three? What are the similarities? Note that this change map is based on pairs of images and not on the full time series as the previous three. How does this affect your results?
</font>
</div>
<br>
<hr>

<hr>
<br>
<font face="Calibri" size="5"> <b> 7. Write Our Change Detection Results and Metrics Images to GeoTIFF files</b> </font>

<font face="Calibri" size="4"> <b> 7.1 Determine Output Geometry </b> </font>

<font face="Calibri" size="3">First, we need to <b>set the correct geotransformation and projection information</b>. We retrieve the values from the input images:
</font> 

In [ ]:
proj = img.GetProjection()
geotrans = list(img.GetGeoTransform())
geotrans

<br>
<font face="Calibri" size="4"> <b> 7.2 Output Time Series Metrics Images </b> </font>

<font face="Calibri" size="3">We use the root of the time series data stack name and append a _ts_metrics_&lt;metric&gt;.tif ending as filenames:
</font> 

In [ ]:
# Time Series Metrics as image image:
# We make a new subdirectory where we will store the images
dirname = image_file_VH.replace('.vrt', '_tsmetrics')
os.makedirs(dirname, exist_ok=True)
print(dirname)

<br>
<font face="Calibri" size="3">Now we can <b>output the individual metrics as GeoTIFF images</b>:
</font> 

In [ ]:
names=[] # List to keep track of all the names
for i in metrics:
    name = os.path.join(dirname, image_file_VH.replace('.vrt','_'+i+'.tif'))
    create_geotiff(name, metrics[i], gdal.GDT_Float32, np.nan,[i], geo_t=geotrans, projection=proj)
    names.append(name)

<br>
<font face="Calibri" size="4"> <b> 7.3 Build a Virtual Raster Table on the Metrics GeoTIFF images </b> </font>

<font face="Calibri" size="3">To tie the images into one new raster stack of time series metrics we build a virtual raster table with all the metrics. 

Trick: Use ' '.join(Names) to build one long string of names separated by a space as input to *gdalbuildvrt*:
</font> 

In [ ]:
cmd = 'gdalbuildvrt -separate -overwrite -vrtnodata nan ' + \
   dirname + '.vrt ' + ' '.join(names)
# print(cmd)
os.system(cmd)

In [ ]:
os.getcwd()

In [ ]:
print('Time Series Metrics VRT File:\n', f"{dirname}.vrt")

<br>
<font face="Calibri" size="4"> <b> 7.4 Create GeoTIFFs for the Change Iamges from our Four Change Detection Attempts </b> </font>

<font face="Calibri" size="3">We are going to write GeoTIFF output files that stores the results from the classifiers:
</font> 

In [ ]:
imagenamepdiff = os.path.join(product_path, image_file_VH.replace('.vrt', '_pdiff_thresholds.tif'))
imagenamevar = os.path.join(product_path, image_file_VH.replace('.vrt', '_var_thresholds.tif'))
imagenamecov = os.path.join(product_path, image_file_VH.replace('.vrt', '_cov_thresholds.tif'))
imagenamelr = os.path.join(product_path,image_file_VH.replace('.vrt', '_lr_thresholds.tif'))

create_geotiff(imagenamepdiff, maskpdiffrange, gdal.GDT_Byte, np.nan, [i], geo_t=geotrans, projection=proj)
create_geotiff(imagenamevar, maskvar, gdal.GDT_Byte, np.nan, [i], geo_t=geotrans, projection=proj)
create_geotiff(imagenamecov, maskcov, gdal.GDT_Byte, np.nan, [i], geo_t=geotrans, projection=proj)
_ = create_geotiff(imagenamelr, masklr, gdal.GDT_Byte, np.nan, [i], geo_t=geotrans, projection=proj)

<hr>
<br>
<font face="Calibri" size="5"> <b> 8. Conclusion</b> </font>

<font face="Calibri" size="3">Thresholds for the three methods are necessarily site dependent and need to be identified with calibration data or visual post-classification interpretation, and can subsequently be adjusted to maximize classification accuracy. Also, some methods will have advantages in different scenarios. 

For a bit more information on change detection and SAR in general, please look at the recently published <a href="https://gis1.servirglobal.net/TrainingMaterials/SAR/SARHB_FullRes.pdf" target="_blank">SAR Handbook: Comprehensive Methodologies for Forest Monitoring and Biomass Estimation</a>.
</font> 

<hr>
<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISES</u> </font></b> 

<font face="Calibri" size="3"> Explore the 78 image deep data stack a bit more. Pick different time steps for the log-ratio change detection to create additional change maps. Answer the following questions for yourself:

<ul>
  <li>Change the threshold and band choices in this notebook to see the effects on detected changes.</li>
  <li>Load created change masks into QGIS and compare the detected areas with your time series plots and image data.</li>
</ul>

</font>
</div>
<br>
<hr>

<font face="Calibri" size="2"> <i>SAR Training Materials - Version 1.1 - June 2019 </i>
</font>